In [1]:
import os
os.chdir("../")
os.getcwd()

'/data1/xhuan192/codes/medagents-benchmark'

In [2]:
from pathlib import Path
import json

In [3]:
# 4o:  2.5 / 1_000_000, 10 / 1_000_000
def parse_result(data):
    num_samples = len(data)
    num_correct = 0
    prompt_tokens = 0
    completion_tokens = 0
    total_time = 0
    for sample in data:
        if sample["predicted_answer"].upper() == sample["answer_idx"].upper():
            num_correct += 1
        prompt_tokens += sample['token_usage']["prompt_tokens"]
        completion_tokens += sample['token_usage']["completion_tokens"]
        total_time += sample["time_elapsed"]
    total_tokens = prompt_tokens + completion_tokens
    avg_prompt_tokens = prompt_tokens / num_samples
    avg_completion_tokens = completion_tokens / num_samples
    avg_tokens = total_tokens / num_samples
    
    accuracy = num_correct / num_samples
    # 4o
    input_cost = prompt_tokens * 2.5 / 1_000_000
    output_cost = completion_tokens * 10 / 1_000_000
    total_cost = input_cost + output_cost

    average_input_cost = input_cost / num_samples
    average_output_cost = output_cost / num_samples
    average_total_cost = total_cost / num_samples

    average_time = total_time / num_samples
    return {
        "accuracy": accuracy,
        "avg_prompt_tokens": avg_prompt_tokens,
        "avg_completion_tokens": avg_completion_tokens,
        "avg_tokens": avg_tokens,
        "input_cost": input_cost,
        "output_cost": output_cost,
        "total_cost": total_cost,
        "average_input_cost": average_input_cost,
        "average_output_cost": average_output_cost,
        "average_total_cost": average_total_cost,
        "total_time": total_time,
        "average_time": average_time,
        "input_tokens": prompt_tokens,
        "output_tokens": completion_tokens,
        "num_samples": num_samples,
    }


In [4]:
output_dir = Path("outputs")
# get all json files recursively in the output_dir
json_files = list(output_dir.rglob("*.json"))
"""
├── mdagents
│   ├── medbullets
│   │   └── gpt-4o-1120-nofilter-global_medbullets_test_hard_adaptive.json
│   └── pubmedqa
│       └── gpt-4o-1120-nofilter-global_pubmedqa_test_hard_adaptive.json
├── medagents
│   ├── medbullets
│   │   └── gpt-4o-1120-nofilter-global-medbullets-test_hard-syn_verif.json
└── medprompt
    ├── cot
    │   ├── medbullets
    │   │   └── gpt-4o-1120-nofilter-global-medbullets-test_hard-cot.json
"""

method_dataset_result_mapping = {}
for json_file in json_files:
    # get the method and dataset from the file name
    method, dataset = json_file.parts[-3], json_file.parts[-2]
    model = json_file.parts[-1].split(dataset)[0]

    if method not in method_dataset_result_mapping:
        method_dataset_result_mapping[method] = {}

    # get the json data
    with open(json_file, "r") as f:
        data = json.load(f)
    method_dataset_result_mapping[method][dataset] = parse_result(data)

In [5]:
def parser_df(method_dataset_result_mapping, key):
    method_dataset_acc_mapping = {}
    for method, dataset_result_mapping in method_dataset_result_mapping.items():
        for dataset, result in dataset_result_mapping.items():
            if method not in method_dataset_acc_mapping:
                method_dataset_acc_mapping[method] = {}
            method_dataset_acc_mapping[method][dataset] = result[key]
    return method_dataset_acc_mapping


In [6]:
import pandas as pd


In [7]:
key = "accuracy"
print(key)
df = pd.DataFrame(parser_df(method_dataset_result_mapping,key))
df = df[["zero_shot",
"few_shot",
"cot",
"cot_sc",
"self_refine",
"multi_persona",
"medprompt",
"medagents",
"mdagents"]]
df = df.transpose()
df["avg"], df["sum"] = df.mean(axis=1), df.sum(axis=1)
df

accuracy


,medbullets,pubmedqa,medxpertqa-r,mmlu,medmcqa,medqa,medexqa,medxpertqa-u,mmlu-pro,avg,sum
zero_shot,0.179775,0.090000,0.130000,0.287671,0.280000,0.420000,0.150000,0.11000,0.370000,0.224161,2.017447
few_shot,0.258427,0.160000,0.140000,0.246575,0.220000,0.350000,0.170000,0.06000,0.160000,0.196111,1.765002
cot,0.314607,0.080000,0.190000,0.301370,0.290000,0.550000,0.250000,0.18000,0.390000,0.282886,2.545977
cot_sc,0.292135,0.100000,0.150000,0.369863,0.290000,0.520000,0.220000,0.18000,0.380000,0.278000,2.501998
self_refine,0.337079,0.150000,NaN,0.333333,0.290000,0.610000,NaN,NaN,NaN,0.344082,1.720412
multi_persona,0.235955,0.110000,0.150000,0.397260,0.310000,0.400000,0.250000,0.12000,0.450000,0.269246,2.423215
medprompt,0.269663,0.130000,0.210000,0.397260,0.300000,0.480000,0.200000,0.22000,0.450000,0.295214,2.656923
medagents,0.222222,0.160000,0.170000,NaN,0.320000,0.510000,0.210000,0.16000,0.380000,0.266528,2.132222
mdagents,0.269231,0.222222,0.105263,0.262295,0.342105,0.326923,0.202703,0.06383,0.268293,0.229207,2.062865


In [8]:
key = "num_samples"
print(key)
df = pd.DataFrame(parser_df(method_dataset_result_mapping,key))
df = df[["zero_shot",
"few_shot",
"cot",
"cot_sc",
"self_refine",
"multi_persona",
"medprompt",
"medagents",
"mdagents"]]
df = df.transpose()
df["avg"], df["sum"] = df.mean(axis=1), df.sum(axis=1)
df

num_samples


,medbullets,pubmedqa,medxpertqa-r,mmlu,medmcqa,medqa,medexqa,medxpertqa-u,mmlu-pro,avg,sum
zero_shot,89.0,100.0,100.0,73.0,100.0,100.0,100.0,100.0,100.0,95.777778,862.0
few_shot,89.0,100.0,100.0,73.0,100.0,100.0,100.0,100.0,100.0,95.777778,862.0
cot,89.0,100.0,100.0,73.0,100.0,100.0,100.0,100.0,100.0,95.777778,862.0
cot_sc,89.0,100.0,100.0,73.0,100.0,100.0,100.0,100.0,100.0,95.777778,862.0
self_refine,89.0,100.0,NaN,6.0,100.0,100.0,NaN,NaN,NaN,79.000000,395.0
multi_persona,89.0,100.0,100.0,73.0,100.0,100.0,100.0,100.0,100.0,95.777778,862.0
medprompt,89.0,100.0,100.0,73.0,100.0,100.0,100.0,100.0,100.0,95.777778,862.0
medagents,63.0,100.0,100.0,NaN,100.0,100.0,100.0,100.0,100.0,95.375000,763.0
mdagents,52.0,54.0,38.0,61.0,76.0,52.0,74.0,47.0,41.0,55.000000,495.0


In [9]:
key = "total_cost"
print(key)
df = pd.DataFrame(parser_df(method_dataset_result_mapping,key))
df = df[["zero_shot",
"few_shot",
"cot",
"cot_sc",
"self_refine",
"multi_persona",
"medprompt",
"medagents",
"mdagents"]]
df = df.transpose()
df["avg"], df["sum"] = df.mean(axis=1), df.sum(axis=1)
display(df)
print(f"total cost", df.sum().sum())

total_cost


,medbullets,pubmedqa,medxpertqa-r,mmlu,medmcqa,medqa,medexqa,medxpertqa-u,mmlu-pro,avg,sum
zero_shot,0.074565,0.105178,0.114522,0.032890,0.034560,0.076808,0.035950,0.099218,0.058078,0.070196,0.631768
few_shot,0.559813,0.537118,1.205852,0.208873,0.121590,0.555608,0.194170,1.039848,0.308457,0.525703,4.731327
cot,0.662070,0.508057,0.849172,0.349055,0.435080,0.677227,0.424970,0.756698,0.606838,0.585463,5.269168
cot_sc,3.529535,2.718037,4.424685,1.839425,2.307410,3.582337,2.284550,3.943985,3.279315,3.101031,27.909280
self_refine,2.110092,1.811352,NaN,0.067845,1.450770,2.091292,NaN,NaN,NaN,1.506271,7.531353
multi_persona,5.627782,4.823738,7.081045,3.149475,4.074900,5.731768,3.954190,6.439715,5.424280,5.145210,46.306893
medprompt,4.137680,3.711560,6.461752,1.901480,2.257532,4.087738,2.572150,6.023200,3.581277,3.859374,34.734370
medagents,8.965180,13.520955,15.979782,NaN,11.091995,13.729930,10.277535,14.056593,11.695663,12.414704,99.317633
mdagents,15.745920,29.350395,16.971700,2.496965,2.766865,14.814553,2.023792,18.624862,4.325690,11.902305,107.120742


total cost 706.2153226180556


In [10]:
key = "total_time"
print(key)
df = pd.DataFrame(parser_df(method_dataset_result_mapping,key))
df = df[["zero_shot",
"few_shot",
"cot",
"cot_sc",
"self_refine",
"multi_persona",
"medprompt",
"medagents",
"mdagents"]]
df = df.transpose()
df["avg"], df["sum"] = df.mean(axis=1), df.sum(axis=1)
display(df)
print(f"total time (sequential)", df.sum().sum()/ 3600, "hours")

total_time


,medbullets,pubmedqa,medxpertqa-r,mmlu,medmcqa,medqa,medexqa,medxpertqa-u,mmlu-pro,avg,sum
zero_shot,117.423813,120.036392,130.322861,91.745115,122.114668,159.455000,133.207150,120.110625,140.575729,126.110150,1134.991352
few_shot,431.876560,182.442526,497.295514,146.547857,149.204682,416.124951,189.426589,306.918099,223.121611,282.550932,2542.958389
cot,741.143786,577.054410,911.187562,432.599729,562.309358,775.587569,554.230422,833.257928,728.123208,679.499330,6115.493973
cot_sc,4086.096926,3069.240881,4610.132303,2296.453717,2999.074553,4261.197869,2885.214147,4114.840982,3948.115543,3585.596324,32270.366920
self_refine,2740.980017,1873.401011,NaN,155.648026,1870.088043,2188.321566,NaN,NaN,NaN,1765.687733,8828.438664
multi_persona,6354.858115,5487.516438,7836.027494,3972.571823,5140.385920,6927.533108,5104.391573,7163.115247,6645.460177,6070.206655,54631.859897
medprompt,2691.566365,2186.517701,3535.895233,1498.090172,1938.659024,2742.306511,2091.636876,3197.637639,2629.798860,2501.345376,22512.108380
medagents,6195.258976,7909.034654,10008.023817,NaN,8386.952224,8603.323901,7670.366319,8575.908013,8216.173803,8195.630213,65565.041707
mdagents,5782.756743,9974.618224,4881.718329,1349.963703,1810.389738,5082.428447,1511.949758,4631.696717,1705.182806,4081.189385,36730.704467


total time (sequential) 135.5421509987782 hours


In [11]:
key = "total_time"
print(key)
df = pd.DataFrame(parser_df(method_dataset_result_mapping,key))
df = df[["zero_shot",
"few_shot",
"cot",
"cot_sc",
"self_refine",
"multi_persona",
"medprompt",
"medagents",
"mdagents"]]
df = df.transpose()
df["avg"], df["sum"] = df.mean(axis=1), df.sum(axis=1)
display(df)
print(f"total time (sequential)", df.sum().sum()/ 3600, "hours")

total_time


,medbullets,pubmedqa,medxpertqa-r,mmlu,medmcqa,medqa,medexqa,medxpertqa-u,mmlu-pro,avg,sum
zero_shot,117.423813,120.036392,130.322861,91.745115,122.114668,159.455000,133.207150,120.110625,140.575729,126.110150,1134.991352
few_shot,431.876560,182.442526,497.295514,146.547857,149.204682,416.124951,189.426589,306.918099,223.121611,282.550932,2542.958389
cot,741.143786,577.054410,911.187562,432.599729,562.309358,775.587569,554.230422,833.257928,728.123208,679.499330,6115.493973
cot_sc,4086.096926,3069.240881,4610.132303,2296.453717,2999.074553,4261.197869,2885.214147,4114.840982,3948.115543,3585.596324,32270.366920
self_refine,2740.980017,1873.401011,NaN,155.648026,1870.088043,2188.321566,NaN,NaN,NaN,1765.687733,8828.438664
multi_persona,6354.858115,5487.516438,7836.027494,3972.571823,5140.385920,6927.533108,5104.391573,7163.115247,6645.460177,6070.206655,54631.859897
medprompt,2691.566365,2186.517701,3535.895233,1498.090172,1938.659024,2742.306511,2091.636876,3197.637639,2629.798860,2501.345376,22512.108380
medagents,6195.258976,7909.034654,10008.023817,NaN,8386.952224,8603.323901,7670.366319,8575.908013,8216.173803,8195.630213,65565.041707
mdagents,5782.756743,9974.618224,4881.718329,1349.963703,1810.389738,5082.428447,1511.949758,4631.696717,1705.182806,4081.189385,36730.704467


total time (sequential) 135.5421509987782 hours


In [12]:
key = "avg_tokens"
print(key)
df = pd.DataFrame(parser_df(method_dataset_result_mapping,key))
df = df[["zero_shot",
"few_shot",
"cot",
"cot_sc",
"self_refine",
"multi_persona",
"medprompt",
"medagents",
"mdagents"]]
df = df.transpose()
df["avg"], df["sum"] = df.mean(axis=1), df.sum(axis=1)
display(df)
print(f"avg_tokens", df.sum().sum())

avg_tokens


,medbullets,pubmedqa,medxpertqa-r,mmlu,medmcqa,medqa,medexqa,medxpertqa-u,mmlu-pro,avg,sum
zero_shot,327.471910,413.630000,449.390000,172.986301,128.370000,299.460000,136.510000,388.080000,224.390000,282.254246,2540.288211
few_shot,1504.943820,1997.570000,3884.470000,921.808219,402.150000,1410.510000,577.480000,3672.460000,982.520000,1705.990227,15353.912039
cot,998.089888,827.010000,1194.540000,616.589041,539.390000,910.380000,536.030000,1056.060000,783.650000,829.082103,7461.738929
cot_sc,5534.415730,4675.170000,6540.390000,3393.767123,2976.740000,5024.760000,2994.410000,5783.660000,4383.660000,4589.663650,41306.972854
self_refine,3850.707865,3164.150000,NaN,1675.000000,2136.450000,3383.310000,NaN,NaN,NaN,2841.923573,14209.617865
multi_persona,8249.966292,7199.610000,9647.470000,5438.095890,4960.410000,7512.260000,4884.970000,8683.340000,6811.900000,7043.113576,63388.022183
medprompt,11805.359551,10580.150000,17922.240000,6398.000000,5492.320000,10456.220000,6270.850000,16753.990000,8687.930000,10485.228839,94367.059551
medagents,34678.539683,34092.060000,39613.370000,NaN,25692.260000,33455.770000,23995.980000,34452.340000,27710.720000,31711.379960,253691.039683
mdagents,107502.173077,203644.055556,163944.815789,13325.540984,11608.355263,101831.519231,8243.675676,146962.276596,36705.121951,88196.392680,793767.534122


avg_tokens 2719857.3997266684
